In [1]:
import pandas as pd 
from matplotlib import pyplot as plt 
import seaborn as sns 

In [2]:
# TF detector
def TF_detector(df_rolling):
    '''
    parameters: dataframe
    <시작 조건>
    시작 조건 0. TF 중 = False
    시작 조건 1. time > 이전 TF 종료 시간
    시작 조건 2. HDD >= {HDD_threshold=500} (time - )
    시작 조건 3. FB > 0, time - {possible_time_variance=1}
    시작 조건 4. HDD >= {HDD_threshold=500} 시간 이후, first FB=False 상태로 HDD < {HDD_lull_cut=50} 생기면 이전 HDD time 제외

    <종료 조건>
    종료 조건 0. TF 진행 중 = True
    종료 조건 1. FB = 0 이 {no_FB_duration=10}초 이상
    종료 조건 2. time = ts_map_end_time

    *stagger
    *길어지는 한타
    FB 0 되고 10s동안 추가 킬이 없어야 한타 종료로 인정

    return:     TF_timestamps: dataframe of [['match_id', 'map_num', 'TF_start_time_stamps', 'TF_end_time_stamps']]
    '''
    global is_TF_ongoing
    is_TF_ongoing = False
    global FB_happened
    FB_happened = False
    global FB_silence
    FB_silence = False

    TF_start_time_stamps = []
    TF_end_time_stamps = []

    HDD_threshold = 500
    HDD_lull_cut = 500
    FB_threshold = 0
    possible_time_variance = 2
    no_FB_duration = 10

    # start condition
    def TF_start(idx):
        global is_TF_ongoing
        global FB_happened
        global FB_silence
        
        def condition0(): # TF 중 = False
            global is_TF_ongoing
            if is_TF_ongoing == True:
                cond0 = False
            else:
                cond0 = True
            return cond0

        def condition1(idx): # time > 이전 TF 종료 시간
            if (idx < df_rolling.index[-1]) == True:
                cond1 = True
            else:
                cond1 = False
            return cond1

        def condition2(idx): # HDD >= {HDD_threshold}
            if df_rolling.loc[idx, 'HDD'] > HDD_threshold:
                cond2 = True
            else:
                cond2 = False
            return cond2 
        
        def condition3(idx): # FB > 0
            global FB_happened
            if df_rolling.loc[idx, 'FB'] > FB_threshold:
                cond3 = True
                FB_happened = True # toggle on
            else:
                cond3 = False
            return cond3
        
        def condition4(idx): # HDD가 오르는 추세에 있어야
            if (df_rolling.loc[idx - no_FB_duration : idx, 'HDD'].mean() <= df_rolling.loc[idx : idx + no_FB_duration, 'HDD'].mean()) == True:
                cond4 = True
            else:
                cond4 = False
            return cond4 
        
        def condition5(idx): # HDD 가 {no_FB_duration} 범위에서 최소값일 경우
            if (df_rolling.loc[idx, 'HDD'] <= df_rolling.loc[idx : idx + no_FB_duration, 'HDD'].min()) == True:
                cond5 = True
            else:
                cond5 = False
            return cond5 


        # check if start_condition == True
        start_condition = condition0() & condition1(idx) & ( (condition2(idx) & condition4(idx) & condition5(idx)) | condition3(idx) ) 
        
        if start_condition == True:
            is_TF_ongoing = True # toggle on
            FB_silence = False # toggle off

        return is_TF_ongoing
    
    # end condition
    def TF_end(idx):
        global is_TF_ongoing
        global FB_happened
        global FB_silence

        def condition0(): # TF 진행 중 = True
            global is_TF_ongoing
            if is_TF_ongoing == True:
                cond0 = True
            else:
                cond0 = False 
            return cond0

        def condition1(idx): # FB = 0 이 {no_FB_duration}초 이상 유지될 때
            global FB_silence
            if df_rolling.loc[idx: idx + no_FB_duration, 'FB'].sum() == 0:
                cond1 = True 
                FB_silence = True # toggle on
            else:
                cond1 = False
            return cond1 
        
        def condition2(idx): # time = ts_map_end_time (Map이 종료됐을 때)
            if idx == df_rolling.index[-1]:
                cond2 = True
            else:
                cond2 = False
            return cond2
        
        def condition3(idx): # FB > 0
            global FB_happened
            if df_rolling.loc[idx, 'FB'] > FB_threshold:
                cond3 = True
                FB_happened = True # toggle on
            else:
                cond3 = False
            return cond3
        
        def condition4(idx): # while FB == 0 & HDD <= {HDD_lull_cut}
            if (df_rolling.loc[idx, 'FB'] == 0) & (df_rolling.loc[idx, 'HDD'] <= HDD_lull_cut):
                cond4 = True
            else:
                cond4 = False
            return cond4
        
        def condition5(idx): # 2s 동안 mean(HDD) < HDD_threshold 
            if df_rolling.loc[idx:idx + 2, 'HDD'].mean() < HDD_threshold:
                cond5 = True
            else:
                cond5 = False
            return cond5

        # init FB_happened
        condition3(idx)

        # check if end_condition == True
        end_condition = ( condition0() & ((condition1(idx) & condition5(idx)) & FB_happened) | (FB_happened & condition4(idx)) ) | condition2(idx)

        if end_condition == True:
            is_TF_ongoing = False # toggle off
            FB_happened = False # toggle off

        return is_TF_ongoing

    TF_status = []

    for idx in df_rolling.index:
        if is_TF_ongoing == False:
            TF_status.append(TF_start(idx))
        else:
            TF_status.append(TF_end(idx))

    df_rolling['TF_status'] = TF_status

    return df_rolling # [['RCP', 'HDD', 'FB', 'TF_status']], index=UTC timestamp

In [3]:
# get TF_time_range
def get_true_range(df=None, column='TF_status'):
    '''
    parameters: data: dataframe with timestamp index
                column: target column to arrange
    
    returns:    index tuple
    '''
    range_list = []
    prev_val = False

    for inx, val in df[column].iteritems():
        if prev_val != val:
            if val:
                start = inx
            else:
                range_list.append((start, inx))

        prev_inx = inx
        prev_val = val
    
    TF_time_range = pd.DataFrame(range_list, columns=['TF_start_time', 'TF_end_time'])
    
    return TF_time_range # df[['TF_start_time', 'TF_end_time']]

In [62]:
class ScrimLog():
    def __init__(self, match_id=None):
        self.match_id = match_id # match_id = filename
        self.set_directory()
        self.set_df_merge()

    def set_directory(self):
        path = 'C:/Users/jyc13/Documents/Overwatch/Workshop'
        filename = self.match_id + '.csv'
        filepath = os.path.join(path, filename)
        self.df_init = pd.read_csv(filepath)
        self.df_init['Match_id'] = self.match_id
    
    def set_df_merge(self):
        grouping = ['Match_id', 'Map', 'Section', 'Timestamp', 'Team', 'Player', 'Hero']
        df_group = self.df_init.groupby(grouping).sum()
        dx = df_group.groupby([x for x in grouping if x != 'Timestamp']).diff().fillna(0)
        df_merge = pd.merge(df_group, dx, how='outer', left_index=True, right_index=True, suffixes=('', '/s'))

        # add Alives
        section_list = df_merge.reset_index('Section')['Section'].unique()
        player_list = df_merge.reset_index('Player')['Player'].unique()

        # set rolling df by moving window Multiindex=['Map', 'Section', 'Team', 'Timestamp']
        rolling = pd.DataFrame()

        for section in section_list:
            for player in player_list:
                df_tmp = df_merge.xs((section, player), level=['Section', 'Player'], drop_level=False)
                rolling_tmp = df_tmp.groupby(by=['Map', 'Section','Team', 'Timestamp']).sum()
                rolling_tmp = rolling_tmp.rolling(window=10).sum().fillna(rolling_tmp) # default window=10 sec
                rolling = pd.concat([rolling, rolling_tmp])

        alives = 6 - rolling.groupby(by=['Map', 'Section', 'Timestamp', 'Team']).sum()[['Deaths/s']]
        alives.rename(columns = {'Deaths/s':'Alives'}, inplace=True)
        team_one_name = self.df_init['Team'].unique()[0]
        team_two_name = self.df_init['Team'].unique()[1]

        self.df_init['Team_one_name'] = team_one_name 
        self.df_init['Team_two_name'] = team_two_name

        X = alives.xs(team_one_name, level='Team', drop_level=False)
        Y = alives.xs(team_two_name, level='Team', drop_level=False)

        # add RCP
        def Alives(X, Y):
            df_merge = pd.merge(X, Y, how='outer', on=[x for x in X.index.names if x != 'Team'], suffixes=(f'_{team_one_name}', f'_{team_two_name}'))
            df_merge['RCP'] = (df_merge[f'Alives_{team_one_name}']**2 - df_merge[f'Alives_{team_two_name}']**2) / (df_merge[[f'Alives_{team_one_name}', f'Alives_{team_two_name}']].max(axis=1)) # Af = (A0^2 - B0^2)/A0
            df_merge['RCP'].fillna(0) # fill nan=0 in case X == Y == 0

            return df_merge    

        Alives = Alives(X, Y)
        df_merge = pd.merge(df_merge, Alives, how='outer', left_index=True, right_index=True)

        ## FB_value and Death risk
        team_one = df_merge.xs(team_one_name, level='Team', drop_level=False)
        team_two = df_merge.xs(team_two_name, level='Team', drop_level=False)

        # RCP
        def RCP(X, Y):
            RCP = (X**2 - Y**2).div(X)
            RCP = RCP.fillna(-Y) # fill nan = -Y in case X == 0
            return RCP

        # add FB value
        def FB_value(X, Y, FB):
            FB_value = RCP(X, Y - FB) - RCP(X, Y)
            return FB_value 
        
        def Death_risk(X, Y, Death):
            Death_risk = RCP(X, Y) - RCP(X - Death, Y)
            return Death_risk
        
        # FB value
        team_one_FB_value = FB_value(team_one[f'Alives_{team_one_name}'],  team_one[f'Alives_{team_two_name}'], team_one['Final Blows/s'])
        team_two_FB_value = FB_value(team_two[f'Alives_{team_two_name}'],  team_two[f'Alives_{team_one_name}'], team_two['Final Blows/s'])

        FB_value = pd.concat([team_one_FB_value, team_two_FB_value]).groupby(by=grouping).max()
        FB_value.rename('FB_value', inplace=True)

        # Death risk
        team_one_Death_risk = Death_risk(team_one[f'Alives_{team_one_name}'],  team_one[f'Alives_{team_two_name}'], team_one['Deaths/s'])
        team_two_Death_risk = Death_risk(team_two[f'Alives_{team_two_name}'],  team_two[f'Alives_{team_one_name}'], team_two['Deaths/s'])

        Death_risk = pd.concat([team_one_Death_risk, team_two_Death_risk]).groupby(by=grouping).max()
        Death_risk.rename('Death_risk', inplace=True)

        df_merge = pd.merge(df_merge, FB_value, how='outer', left_index=True, right_index=True)
        df_merge = pd.merge(df_merge, Death_risk, how='outer', left_index=True, right_index=True)

        # get TF_info
        def TF_info(df_merge):
            df_merge = df_merge.reset_index()
            section_list = df_merge['Section'].unique()
            df_merge = df_merge.groupby(by=['Match_id', 'Map', 'Section', 'Timestamp', 'Team', 'Player', 'Hero']).sum()

            TF_rolling = pd.DataFrame()
            for section in section_list:
                df_tmp = df_merge.xs(section, level='Section', drop_level=False)
                TF_rolling_tmp = df_tmp.groupby(by=['Match_id', 'Map', 'Section', 'Timestamp']).sum()
                TF_rolling_tmp = TF_rolling_tmp.rolling(window=10, center=True).sum().fillna(TF_rolling_tmp) # default window=10 sec
                TF_rolling = pd.concat([TF_rolling, TF_rolling_tmp])

            TF_rolling = TF_rolling.groupby(by=['Match_id', 'Map', 'Section', 'Timestamp']).sum()[['Hero Damage Dealt/s', 'Final Blows/s']]
            TF_rolling.rename(columns={'Hero Damage Dealt/s':'HDD', 'Final Blows/s':'FB'}, inplace=True)

            RCP = df_merge.groupby(by=['Match_id', 'Map', 'Section', 'Timestamp']).max()[['RCP']]

            TF_rolling = pd.merge(TF_rolling, RCP, how='outer', left_index=True, right_index=True)

            # df_merge = df_merge.reset_index()
            # df_merge = df_merge.set_index('Timestamp')
            
            TF_info = pd.DataFrame()
            for section in section_list:
                TF_rolling_tmp = TF_rolling.xs(section, level='Section', drop_level=False)
                TF_rolling_tmp = TF_rolling_tmp.reset_index()
                TF_rolling_tmp = TF_rolling_tmp.set_index('Timestamp')
                df_TF = TF_detector(TF_rolling_tmp)
                TF_time_range = get_true_range(df_TF)

                # define TF winner and RCP
                TF_winner_list = []
                TF_order_list = []
                TF_duration_list = []
                TF_RCP_list = []
                TF_order = 0
                
                for idx in TF_time_range.index:
                    TF_order += 1
                    TF_order_list.append(TF_order)
                    TF_duration_list.append((TF_time_range.loc[idx, 'TF_end_time'] - TF_time_range.loc[idx, 'TF_start_time'])) # TF duration in s
                    TF_RCP_sum = TF_rolling_tmp.loc[TF_time_range.loc[idx, 'TF_start_time']:TF_time_range.loc[idx, 'TF_end_time'], 'RCP'].sum()
                    TF_RCP_list.append(TF_RCP_sum) # RCP
                    if TF_RCP_sum > 0:
                        TF_winner_list.append(team_one_name)
                    elif TF_RCP_sum == 0:
                        TF_winner_list.append('draw')
                    else:
                        TF_winner_list.append(team_two_name)
                TF_time_range['TF_order'] = TF_order_list
                TF_time_range['TF_winner'] = TF_winner_list
                TF_time_range['TF_duration'] = TF_duration_list
                TF_time_range['TF_RCP_sum'] = TF_RCP_list

                TF_time_range['Match_id'] = self.match_id
                TF_time_range['Map'] = self.df_init['Map'].unique()[0]
                TF_time_range['Section'] = section

                TF_info = pd.concat([TF_info, TF_time_range])
            
            return TF_info

        self.TF_info = TF_info(df_merge)

        # add TF_info
        tmp_merge = pd.DataFrame()
        for section in section_list:
            tmp = df_merge.xs(section, level='Section', drop_level=False)
            tmp = tmp.reset_index().set_index('Timestamp')
            TF_info_tmp = self.TF_info[self.TF_info['Section'] == section]
            for idx in TF_info_tmp.index:
                start, end = TF_info_tmp.loc[idx, 'TF_start_time'], TF_info_tmp.loc[idx, 'TF_end_time']
                tmp.loc[start:end, 'TF_order'] = TF_info_tmp.loc[idx, 'TF_order']
                tmp.loc[start:end, 'TF_winner'] = TF_info_tmp.loc[idx, 'TF_winner']
                tmp.loc[start:end, 'TF_duration'] = TF_info_tmp.loc[idx, 'TF_duration']
                tmp.loc[start:end, 'TF_RCP_sum'] = TF_info_tmp.loc[idx, 'TF_RCP_sum']
            
            tmp_merge = pd.concat([tmp_merge, tmp])
        
        df_merge = tmp_merge

        # reset index and grouping
        def indexing(df):
            df = df.reset_index()
            df = df.groupby(by=grouping).max()
            return df 
        
        self.df_merge = indexing(df_merge)
    
    def get_df_merge(self):
        return self.df_merge
    
    def get_TF_info(self):
        return self.TF_info

    


In [63]:
scrim = ScrimLog('20210318_02_Oasis_NYE_vs_O2B')
df_merge = scrim.get_df_merge()
display(df_merge)

Hero Damage Dealt  \
Match_id                     Map   Section Timestamp Team     Player  Hero                              
20210318_02_Oasis_NYE_vs_O2B Oasis 0       0         NYXL     NYE001  Lucio                      0.00   
                                                              NYE004  Baptiste                   0.00   
                                                              NYE012  Symmetra                   0.00   
                                                              NYE013  D.Va                       0.00   
                                                              NYE014  Mei                        0.00   
...                                                                                               ...   
                                   2       229       O2 Blast Kilo    McCree                  9236.00   
                                                              MAX     D.Va                    7109.15   
                                                              Proper  Tracer                  8110.06   
                                                              Vindaim Lucio                   3953.39   
                                                              junbin  WreckingBall            8281.36   

                                                                                    Barrier Damage Dealt  \
Match_id                     Map   Section Timestamp Team     Player  Hero                                 
20210318_02_Oasis_NYE_vs_O2B Oasis 0       0         NYXL     NYE001  Lucio                         0.00   
                                                              NYE004  Baptiste                      0.00   
                                                              NYE012  Symmetra                      0.00   
                                                              NYE013  D.Va                          0.00   
                                                              NYE014  Mei                           0.00   
...                                                                                                  ...   
                                   2       229       O2 Blast Kilo    McCree                     2449.69   
                                                              MAX     D.Va                       4384.07   
                                                              Proper  Tracer                     2458.82   
                                                              Vindaim Lucio                      1535.26   
                                                              junbin  WreckingBall               3680.28   

                                                                                    Damage Blocked  \
Match_id                     Map   Section Timestamp Team     Player  Hero                           
20210318_02_Oasis_NYE_vs_O2B Oasis 0       0         NYXL     NYE001  Lucio                   0.00   
                                                              NYE004  Baptiste                0.00   
                                                              NYE012  Symmetra                0.00   
                                                              NYE013  D.Va                    0.00   
                                                              NYE014  Mei                     0.00   
...                                                                                            ...   
                                   2       229       O2 Blast Kilo    McCree                  0.00   
                                                              MAX     D.Va                 6053.22   
                                                              Proper  Tracer               3352.87   
                                                              Vindaim Lucio                1199.55   
                                                              junbin  WreckingBall       

In [6]:
# save
df_merge.to_csv(f'results/{scrim.match_id}.csv')